In [ ]:
# # !pip install datasets
# !pip install tokenizers
# # !pip install transformers

# !pip install datasets transformers==4.28.0

In [ ]:
!pip install datasets

In [ ]:
!git clone --single-branch --branch fast_tokenizers_BARTpho_PhoBERT_BERTweet https://github.com/datquocnguyen/transformers.git

fatal: destination path 'transformers' already exists and is not an empty directory.


In [ ]:
%cd transformers
!pip3 install -e .

/content/transformers
Obtaining file:///content/transformers
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for transformers (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.32.0.dev0-0.editable-py3-none-any.whl size=38186 sha256=b1a3a2a1866dbb15655487626321e6ebf3da4e11a6741f709fcba6a5fda3955a
  Stored in directory: /tmp/pip-ephem-wheel-cache-2ybegcew/wheels/7c/35/80/e946b22a081210c6642e607ed65b2a5b9a4d9259695ee2caf5
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.32.0.dev0
    Uninstalling transformers-4.32.0.dev0:
      Successfully uninstalled transformers-4.32.0.dev0


In [ ]:
!pip install accelerate -U

In [ ]:
!pip3 install tokenizers

In [ ]:
!pip install seqeval

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
from transformers import DataCollatorWithPadding,AutoModelForSequenceClassification, Trainer, TrainingArguments,AutoTokenizer,AutoModel,AutoConfig, DataCollatorForTokenClassification
from transformers.modeling_outputs import TokenClassifierOutput
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import datasets
from datasets import load_dataset,Dataset,DatasetDict,ClassLabel,Sequence

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load and process custom dataset

In [ ]:
df = pd.read_csv('/content/generated_data.csv')

In [ ]:
import ast

# Function to process the label column
def process_label(label):
    # Use ast.literal_eval to safely evaluate the content inside the brackets
    try:
        label_list = ast.literal_eval(label)
        if isinstance(label_list, list):
            if len(label_list) == 0:
                return [0]
            return label_list
    except (SyntaxError, ValueError):
        pass
    return [0]

In [ ]:
df['content'] = df['content'].str.replace(r"[\[\]']", '', regex=True)

In [ ]:
df['content'] = df['content'].apply(lambda x: x.split(" "))

In [ ]:
# Apply the process_label function to each row in the label column
df['NER'] = df['NER'].apply(lambda x: x.replace("[[", "["))
df['NER'] = df['NER'].apply(lambda x: x.replace("]]", "]"))
df['NER'] = df['NER'].apply(process_label)


# Explode the label lists into separate rows
# df = df.explode('label', ignore_index=True)
# df = df.drop(['Unnamed: 0'], axis=1)

In [ ]:
df.rename(columns = {'content': 'tokens', 'NER':'ner_tags'}, inplace = True)

In [ ]:
df = df.drop(['Unnamed: 0', 'label'], axis=1)

In [ ]:
data = Dataset.from_pandas(df)

In [ ]:
data.features

{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}

In [ ]:
class_label = ["O", "B-Subject", "I-Subject", "B-Object", "I-Object", "B-Aspect", "I-Aspect", "B-Predicate", "I-Predicate"]
data = data.cast_column("ner_tags", datasets.Sequence(datasets.ClassLabel(names=class_label)))

Casting the dataset:   0%|          | 0/63376 [00:00<?, ? examples/s]

In [ ]:
train_testvalid = data.train_test_split(test_size=0.2,seed=15)

# Split the 10% test + valid in half test, half valid
test_valid = train_testvalid['test'].train_test_split(test_size=0.5,seed=15)
dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'validation': test_valid['train']})

In [ ]:
dataset["train"].features

{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-Subject', 'I-Subject', 'B-Object', 'I-Object', 'B-Aspect', 'I-Aspect', 'B-Predicate', 'I-Predicate'], id=None), length=-1, id=None)}

In [ ]:
label_names = dataset["train"].features["ner_tags"].feature.names

# Huggingface preprocessing

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2", use_fast = True)

#Get the values for input_ids, token_type_ids, attention_mask
def tokenize_adjust_labels(all_samples_per_split):
  tokenized_samples = tokenizer.batch_encode_plus(all_samples_per_split["tokens"], is_split_into_words=True)
  #tokenized_samples is not a datasets object so this alone won't work with Trainer API, hence map is used
  #so the new keys [input_ids, labels (after adjustment)]
  #can be added to the datasets dict for each train test validation split
  total_adjusted_labels = []
  for k in range(0, len(tokenized_samples["input_ids"])):
    prev_wid = -1
    word_ids_list = tokenized_samples.word_ids(batch_index=k)
    existing_label_ids = all_samples_per_split["ner_tags"][k]
    i = -1
    adjusted_label_ids = []

    for wid in word_ids_list:
      if(wid is None):
        adjusted_label_ids.append(-100)
      elif(wid!=prev_wid):
        i = i + 1
        adjusted_label_ids.append(existing_label_ids[i])
        prev_wid = wid
      else:
        label_name = label_names[existing_label_ids[i]]
        adjusted_label_ids.append(existing_label_ids[i])

    total_adjusted_labels.append(adjusted_label_ids)
  tokenized_samples["labels"] = total_adjusted_labels
  return tokenized_samples

tokenized_dataset = dataset.map(tokenize_adjust_labels, batched=True)

(…)phobert-base-v2/resolve/main/config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

(…)i/phobert-base-v2/resolve/main/vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

(…)i/phobert-base-v2/resolve/main/bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

(…)bert-base-v2/resolve/main/tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

Map:   0%|          | 0/50700 [00:00<?, ? examples/s]

Map:   0%|          | 0/6338 [00:00<?, ? examples/s]

Map:   0%|          | 0/6338 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
import numpy as np
from datasets import load_metric

In [ ]:
metric = load_metric("seqeval")

<ipython-input-26-e20ba34f8cc7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [ ]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    flattened_results = {
        "overall_precision": results["overall_precision"],
        "overall_recall": results["overall_recall"],
        "overall_f1": results["overall_f1"],
        "overall_accuracy": results["overall_accuracy"],
    }
    for k in results.keys():
      if(k not in flattened_results.keys()):
        flattened_results[k+"_f1"]=results[k]["f1"]

    return flattened_results

In [ ]:
id2label = {0: "O", 1: "B-Subject", 2: "I-Subject", 3: "B-Object", 4: "I-Object", 5: "B-Aspect", 6: "B-Aspect",
            7: "B-Predicate", 8: "I-Predicate"}

label2id = {"O": 0 , "B-Subject": 1, "I-Subject": 2 , "B-Object": 3, "I-Object": 4, "B-Aspect": 5, "B-Aspect": 6,
            "B-Predicate": 7, "I-Predicate": 8}

In [ ]:
model = AutoModelForTokenClassification.from_pretrained("vinai/phobert-base-v2", num_labels=len(label_names), id2label=id2label, label2id=label2id)


pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
for name, module in model.named_modules():
  print(name)

In [ ]:
for name, module in model.named_modules():
  freeze_list = [".embeddings", ".layer.0.", ".layer.1.", ".layer.2.", ".layer.3.", ".layer.4.", ".layer.5.", ".layer.6."]
  for layer in freeze_list:
    if layer in name:
      for param in module.parameters():
        param.requires_grad = False

In [ ]:
for name, module in model.named_modules():
    for param in module.parameters():
        if param.requires_grad == True:
          print(name)
          break

In [ ]:
model.num_parameters(only_trainable = True)

134411525

In [ ]:
model.num_parameters(only_trainable = False)

134411525

In [ ]:
batch_size = 128
training_args = TrainingArguments(
output_dir="results",
num_train_epochs=15,
per_device_train_batch_size=batch_size,gio
per_device_eval_batch_size=batch_size,
evaluation_strategy="epoch",
disable_tqdm=False,
logging_steps=len(tokenized_dataset['train']) // batch_size)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

/content/transformers/src/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a PhobertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Overall Precision,Overall Recall,Overall F1,Overall Accuracy,Object F1,Subject F1
1,0.149900,0.022055,0.993266,0.993389,0.993328,0.996464,0.993219,0.993447


In [ ]:
trainer.save_model("/content/drive/MyDrive/VLSP23/Data/HH/baseline_models/phobert_new_data")

In [ ]:
import matplotlib.pyplot as plt

loss_list = []
val_loss_list = []
for i in range(0, len(trainer.state.log_history)):
  if "eval_loss" in trainer.state.log_history[i].keys():
    val_loss_list.append(trainer.state.log_history[i]["eval_loss"])
  if "loss" in trainer.state.log_history[i].keys():
    loss_list.append(trainer.state.log_history[i]["loss"])
  if "eval_loss" in trainer.state.log_history[i].keys() and "loss" in trainer.state.log_history[i].keys():
    print(trainer.state.log_history[i])

In [ ]:
plt.plot(loss_list)

In [ ]:
plt.plot(val_loss_list)

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_dataset["test"])

predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)

true_predictions = [

[label_names[p] for (p, l) in zip(prediction, label) if l != -100]

for prediction, label in zip(predictions, labels)

]

true_labels = [

[label_names[l] for (p, l) in zip(prediction, label) if l != -100]

for prediction, label in zip(predictions, labels)

]

results = metric.compute(predictions=true_predictions, references=true_labels)

results

In [ ]:
from transformers import pipeline

model_test = AutoModelForTokenClassification.from_pretrained("/content/transformers/test_phobert_ner")
nlp = pipeline("token-classification", model=model_test, tokenizer=tokenizer)
example = "Cả hai đều sử dụng công nghệ tấm nên OLED mới nhất của Samsung."
ner_results = nlp(example)

In [ ]:
ner_results